In [76]:
%matplotlib inline
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from patsy import dmatrices
from sklearn import metrics
from statsmodels.stats.outliers_influence import OLSInfluence
import seaborn as sns
import matplotlib.pyplot as plt

In [77]:
auto = pd.read_csv("Auto.csv")
auto["origin"] = auto["origin"].astype("category")
auto["horsepower"] = pd.to_numeric(auto["horsepower"], errors="coerce")
auto["weight"] = pd.to_numeric(auto["weight"], errors="coerce")
auto["cylinders"] = pd.to_numeric(auto["cylinders"], errors="coerce")

In [78]:
# Drop rows with null values
auto = auto.dropna()

In [79]:
def myLinReg(model_formula, df, printMSE=False, seed=42069):
    """
    Function returns the summary for fitted linear model.

    Parameter "model_formula" should be a patsy formula describing the model.
    Parameter "df" is a dataframe.
    """

    # Set seed
    np.random.seed(seed)

    # Split the data into training (80%) and validation set (20%)
    mask = np.random.rand(len(df)) < 0.8
    train = df[mask]
    valid = df[~mask]

    # Prepare the data (dmatrices is from patsy library)
    y_train, X_train = dmatrices(model_formula, data=train, return_type="dataframe")
    y_valid, X_valid = dmatrices(model_formula, data=valid, return_type="dataframe")

    # Train the model
    model = sm.OLS(y_train, X_train)
    result = model.fit()

    train_mse = metrics.mean_squared_error(y_train, result.predict(X_train))
    valid_mse = metrics.mean_squared_error(y_valid, result.predict(X_valid))

    if printMSE == True:
        print(model_formula)
        print(f"MSE_Train: {train_mse}")
        print(f"MSE_Test: {valid_mse}\n")

    # Retrun fitted model summary
    return (result, train_mse, valid_mse)

In [80]:
_ = myLinReg(
    "mpg ~ weight + year + origin",
    auto,
    printMSE=True,
)

mpg ~ weight + year + origin
MSE_Train: 10.984252814363328
MSE_Test: 11.145664511860094



In [81]:
auto_categorical = auto.copy()

# change year to categorical variable
auto_categorical["year"] = auto_categorical["year"].astype("category")

_ = myLinReg(
    "mpg ~ weight + year + origin",
    auto_categorical,
    printMSE=True,
)

mpg ~ weight + year + origin
MSE_Train: 8.845700204567692
MSE_Test: 10.291951880904195



In [82]:
# run 1000 times for different seeds and calculate average MSE
def calculate_average_mse_for_model(model_formula, df):
    MSE_train = []
    MSE_test = []

    for i in range(1000):
        (_, train_mse, valid_mse) = myLinReg(model_formula, df, seed=hash(i))

        MSE_train.append(train_mse)
        MSE_test.append(valid_mse)

    print(f"Average MSE_Train: {np.mean(MSE_train)}")
    print(f"Average MSE_Test: {np.mean(MSE_test)}")

print("Auto dataset:")
calculate_average_mse_for_model("mpg ~ weight + year + origin", auto)
print("\nAuto dataset with year as categorical variable:")
calculate_average_mse_for_model("mpg ~ weight + year + origin", auto_categorical)

Auto dataset:
Average MSE_Train: 10.955305631351523
Average MSE_Test: 11.37273141137678

Auto dataset with year as categorical variable:
Average MSE_Train: 8.97007449803151
Average MSE_Test: 9.9953726104409
